<a href="https://colab.research.google.com/github/francotestori/aauba_02/blob/anto/AA_TP2_Split_train_val_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [ ]:
!wget https://raw.githubusercontent.com/francotestori/aauba_02/d9ac9be49832eaaee67f3dff431a519af4edd23d/resources/audio_digits_with_filename.csv
!wget https://raw.githubusercontent.com/francotestori/aauba_02/master/testing.csv
!wget https://raw.githubusercontent.com/francotestori/aauba_02/master/validation.csv

In [3]:
# Carga de archivos

df = pd.read_csv('audio_digits_with_filename.csv')
testing_list = pd.read_csv('testing.csv')
validation_list = pd.read_csv('validation.csv')

In [ ]:
df.columns

In [ ]:
# Eliminamos columna 
df = df.drop(['Unnamed: 0'],axis=1)

# Creamos una columna que arregla el nombre del audio
df['audio_name'] = df['filename'].str.slice(start=15)

df

In [8]:
# Nos fijamos para cada audio a qué conjunto pertenece

tipo_split = []
for i in df['audio_name'] :
  if i in testing_list['test'].tolist():
    tipo_split.append('test')
  elif i in validation_list['validation'].tolist():
    tipo_split.append('validation') 
  else:
    tipo_split.append('training')

df['split'] = tipo_split

df['split'].value_counts()

training      18620
test           2552
validation     2494
Name: split, dtype: int64

In [ ]:
# Armado de conjuntos

df_test = df[df['split']=='test']
df_val = df[df['split']=='validation']
df_train = df[df['split']=='training']

# Target
y_train = df_train['digit']
y_val = df_val['digit']
y_test = df_test['digit']

# Features
x_train = df_train.drop(['digit','filename','audio_name','split'],axis=1)
x_val = df_val.drop(['digit','filename','audio_name','split'],axis=1)
x_test = df_test.drop(['digit','filename','audio_name','split'],axis=1)